# CSV file before and after datasets

In [ ]:
#this is where the real shit begins

# Cleaning excel sheet and create csv file

In [ ]:
import pandas as pd
import os

file_path = os.path.join(os.getcwd(), "fault_data_inv.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')

# set column names
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)  # dont take the first row

# save as csv
csv_path = "initial_fault_data.csv"
df.to_csv(csv_path, index=False)

print(f"CSV saved at: {csv_path}")


# cleaning csv file

In [ ]:
import pandas as pd

csv_path = "initial_fault_data.csv"
df = pd.read_csv(csv_path)

print(df.head())

# remove values with no change
df_cleaned = df[df.iloc[:, 2:].nunique(axis=1) > 1]

# cleaned csv
cleaned_csv_path = "cleaned_fault_data.csv"
df_cleaned.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned CSV saved at: {cleaned_csv_path}")


# getting mean values and a new csv file

In [ ]:
import pandas as pd

df = pd.read_csv('cleaned_fault_data.csv')

# strip whitespace
df.columns = df.columns.str.strip()

# errors='coerce' will turn non-numeric values into NaN
df.iloc[:, 2:] = df.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')

# mean values before fault (columns C to G)
df['Mean_Before_Fault'] = df.iloc[:, 2:7].mean(axis=1)

# mean values after fault (columns H to L)
df['Mean_After_Fault'] = df.iloc[:, 7:12].mean(axis=1)

# new data
mean_df = df.iloc[:, [0, 1]].copy()
mean_df['Mean_Before_Fault'] = df['Mean_Before_Fault']
mean_df['Mean_After_Fault'] = df['Mean_After_Fault']

# replace NaN values with N/A cause i like it
mean_df = mean_df.fillna("N/A")

# mean values csv
mean_df.to_csv('mean_values.csv', index=False)

# just to see
print(mean_df)


# sort mean_values csv

In [ ]:
import pandas as pd

df = pd.read_csv('mean_values.csv')

df.columns = df.columns.str.strip()

df[['Mean_Before_Fault', 'Mean_After_Fault']] = df[['Mean_Before_Fault', 'Mean_After_Fault']].replace("N/A", pd.NA)
df[['Mean_Before_Fault', 'Mean_After_Fault']] = df[['Mean_Before_Fault', 'Mean_After_Fault']].apply(pd.to_numeric, errors='coerce')

# normal changes
df['Change_Type'] = 'Normal Change'
df.loc[(df['Mean_Before_Fault'] == 0) & (df['Mean_After_Fault'] != 0), 'Change_Type'] = 'Zero to Non-Zero Change'

df['Absolute_Percentage_Change'] = abs((df['Mean_After_Fault'] - df['Mean_Before_Fault']) / df['Mean_Before_Fault']) * 100
df.loc[df['Change_Type'] == 'Zero to Non-Zero Change', 'Absolute_Percentage_Change'] = float('inf')

zero_to_non_zero_df = df[df['Change_Type'] == 'Zero to Non-Zero Change']

normal_change_df = df[df['Change_Type'] != 'Zero to Non-Zero Change']

# absolute change
normal_change_df = normal_change_df.sort_values(by='Absolute_Percentage_Change', ascending=False)

# sort data
sorted_df = pd.concat([zero_to_non_zero_df, normal_change_df], ignore_index=True)

sorted_df = sorted_df.fillna("N/A")

sorted_df.to_csv('mean_values_sorted.csv', index=False)

print(sorted_df)


# time to plot a bar graph to see freaking differences

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('mean_values_sorted.csv')
df.columns = df.columns.str.strip()

num_cols = 10  
num_rows = (len(df.iloc[2:]) // num_cols) + 1  


aspect_ratio = max(num_rows, num_cols)


fig, axes = plt.subplots(num_rows, num_cols, figsize=(aspect_ratio * 3, aspect_ratio * 3))
fig.tight_layout(pad=5.0)
axes = axes.flatten()

for idx, (index, row) in enumerate(df.iloc[2:].iterrows()):
    field_name = row.iloc[0]
    unit = row.iloc[1]
    mean_before = row.iloc[2]
    mean_after = row.iloc[3]
    
    data = {
        'Field Name': ['Before', 'After'],
        'Mean Value': [mean_before, mean_after]
    }
    
    plot_df = pd.DataFrame(data)
    
    ax = axes[idx]
    
    sns.set_theme(style="whitegrid")
    sns.barplot(data=plot_df, x='Field Name', y='Mean Value', hue='Field Name', palette='Blues', ax=ax, legend=False)
    
    ax.set_title(f"{field_name} ({unit})", fontsize=12)
    ax.set_xlabel('')
    ax.set_ylabel(f'{unit}', fontsize=12)
    
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    fontsize=12, color='black', 
                    xytext=(0, 9), textcoords='offset points')
    
    ax.set_xticks([0, 1])
    ax.grid(True, linestyle='--', alpha=0.6)

for idx in range(len(df.iloc[2:]), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


# plot fix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('mean_values.csv')

df.columns = df.columns.str.strip()

row = df.iloc[2] 

field_name = row.iloc[0]  
unit = row.iloc[1]  
mean_before = row.iloc[2]  
mean_after = row.iloc[3]  

data = {
    'Field Name': [f"Before", f"After"],  # x-axis
    'Mean Value': [mean_before, mean_after]
}
plot_df = pd.DataFrame(data)

# plot size
plt.figure(figsize=(10, 6))

sns.set_theme(style="whitegrid")
ax = sns.barplot(data=plot_df, x='Field Name', y='Mean Value', hue='Field Name', palette='Blues', legend=False)

# title and axis labels
ax.set_title(f"{field_name} ({unit})", fontsize=16)
ax.set_xlabel('')  # remove the x-axis title
ax.set_ylabel(f'{unit}', fontsize=12)


for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                fontsize=12, color='black', 
                xytext=(0, 9), textcoords='offset points')


ax.set_xticks([0, 1])

# Add gridlines
ax.grid(True, linestyle='--', alpha=0.6)

# Show the plot
plt.tight_layout()
plt.show()
